# Time Series Analysis of NAICS

<img src="img.svg.png">

The North American Industry Classification System or NAICS is a classification of business establishments by type of economic activity. 
It is used by government and business in Canada, Mexico, and the United States of America.

It has largely replaced the older Standard Industrial Classification (SIC) system, except in some government agencies, such as the U.S. Securities and Exchange Commission (SEC).

An establishment is typically a single physical location, though administratively distinct operations at a single location may be treated as distinct establishments. 
Each establishment is classified to an industry according to the primary business activity taking place there. NAICS does not offer guidance on the classification of enterprises (companies) which are composed of multiple establishments.

NAICS is designed to provide common definitions of the
industrial structure of the three countries and a common statistical framework to facilitate the
analysis of the three economies.

## The data provided contains:

<b>a- Raw data:</b><br>
15 CSV files beginning with RTRA.<br>
These files contain employment data by industry at different levels of aggregation; 2-digit NAICS, 3-digit NAICS, and 4-digit
NAICS. <br>
<ul>
Columns mean as follows:
    <li>
 SYEAR: Survey Year
        </li>
    <li>
 SMTH: Survey Month
        </li>
    <li>
 NAICS: Industry name and associated NAICS code in the bracket
        </li>
    <li>
 _EMPLOYMENT_: Employment
        </li>
</ul>

<b>b- LMO Detailed Industries by NAICS:</b><br> 
An excel file for mapping the RTRA data to the desired data. <br>
The first column of this file has a list of 59 industries that are frequently used.<br>
The second column has their NAICS definitions. <br>
Using these NAICS definitions and RTRA data, you would create a monthly employment data series from 1997 to 2018 for these 59
industries.


<b>c- Data Output Template:</b><br>
An excel file with an empty column for employment. 

## Task

In this task, we need to understand how the NAICS works as a hierarchical structure for defining industries at different levels of aggregation. <br>

<b>For example:</b> <br>
In NAICS 2017 – Statistics Canada.pdf (see page 22), a 2-digit NAICS industry (e.g., 23 - Construction) is
composed of some 3-digit NAICS industries (236 - Construction of buildings, 237 - Heavy
and civil engineering construction, and a few more 3-digit NAICS industries).<br>

Similarly, a 3-digit NAICS industry (e.g., 236 - Construction of buildings), is composed of
4-digit NAICS industries (2361 - Residential building construction and 2362 -Non-residential building construction).

##  Get, and prepare the Dataset:

### a- Loading and exploring the LMO_Detailed_Industries_by_NAICS data:

In [40]:
# import libraries
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as snb
import matplotlib.pyplot as plt

In [41]:
# Loading LMO_Detailed_Industries_by_NAICS data
LMO_Detailed_Industries_df = pd.read_excel("Data\LMO_Detailed_Industries_by_NAICS.xlsx")
LMO_Detailed_Industries_df.head()

,LMO_Detailed_Industry,NAICS
0,Farms,111 & 112
1,"Fishing, hunting and trapping",114
2,Forestry and logging,113
3,Support activities for agriculture and forestry,115
4,Oil and gas extraction,211


In [42]:
# Create a list of NAICS for industries
list_NAICS = LMO_Detailed_Industries_df['NAICS'].astype(str).str.replace(' &', ',').str.split(', ')

In [43]:
list_NAICS.head()

0    [111, 112]
1         [114]
2         [113]
3         [115]
4         [211]
Name: NAICS, dtype: object

In [44]:
LMO_Detailed_Industries_df["list_NAICS"] = list_NAICS

In [45]:
LMO_Detailed_Industries_df.head()

,LMO_Detailed_Industry,NAICS,list_NAICS
0,Farms,111 & 112,"[111, 112]"
1,"Fishing, hunting and trapping",114,[114]
2,Forestry and logging,113,[113]
3,Support activities for agriculture and forestry,115,[115]
4,Oil and gas extraction,211,[211]


### b- Loading and exploring the 2-Digit NAICS Industries data:

In [46]:
# All the csv files have the same columns

In [47]:
# Get the data of 2digit NAICS industries
df_2_NAICS = pd.concat(map(pd.read_csv, ['Data/RTRA_Employ_2NAICS_00_05.csv', 'Data/RTRA_Employ_2NAICS_06_10.csv',
                                         'Data/RTRA_Employ_2NAICS_11_15.csv', 'Data/RTRA_Employ_2NAICS_16_20.csv',
                                          'Data/RTRA_Employ_2NAICS_97_99.csv']))

In [48]:
df_2_NAICS.head()

,SYEAR,SMTH,NAICS,_EMPLOYMENT_
0,2000,1,Accommodation and food services [72],148000
1,2000,1,"Administrative and support, waste management a...",59250
2,2000,1,"Agriculture, forestry, fishing and hunting [11]",61750
3,2000,1,"Arts, entertainment and recreation [71]",39500
4,2000,1,Construction [23],106250


In [49]:
df_2_NAICS.shape

(5472, 4)

### Separate the Industry description and NAICS code 

In [50]:
# Separate the Industry description and NAICS code 
df_2_NAICS['NAICS']= pd.DataFrame(df_2_NAICS['NAICS'].str.split('[').tolist(), columns=["NAICS", "CODE"])
df_2_NAICS['CODE']= pd.DataFrame(df1['CODE'].str.replace(']', ''))

In [51]:
df_2_NAICS

,SYEAR,SMTH,NAICS,_EMPLOYMENT_,CODE
0,2000,1,Accommodation and food services,148000,72
1,2000,1,"Administrative and support, waste management a...",59250,56
2,2000,1,"Agriculture, forestry, fishing and hunting",61750,11
3,2000,1,"Arts, entertainment and recreation",39500,71
4,2000,1,Construction,106250,23
...,...,...,...,...,...
715,1999,12,"Real estate, rental and leasing",37000,53
716,1999,12,Retail trade,230750,44-45
717,1999,12,Transportation and warehousing,117500,48-49
718,1999,12,Utilities,10250,22


In [55]:
# Function to get the 'LMO_Detailed_Industry' values for a NAICS code in RTRA files
def add_lmo_industry(df):
    lmo_df = lmo_detailed_industries_data.apply(lambda y: y["Detailed_Industry"] 
                                                if ((df['code'] in y['NAICS_list'])) else np.nan, axis=1)
    lmo_df = lmo_df.dropna(how='all', axis=0)
    if lmo_df.empty:
        lmo_df = np.nan
    else:
        lmo_df = lmo_df.to_string(index=False)
    return lmo_df

In [56]:
# Get the LMO_Detailed_Industry for the 2digit NAICS RTRA file
df_2_NAICS["Detailed_Industry"] = df_2_NAICS.apply(add_lmo_industry, axis=1)
df_2_NAICS.head(10)

NameError: name 'lmo_detailed_industries_data' is not defined